# SGD Opportunistic Example

SGD Example our goal is:

- Show the power of the opportunistic optimizer

![SGD PLAN](files/sgd_plan.png)

---

## Preparing dependancies

The coursierapi is necessary at import time because it is mandatory to include the maven repository from the local folder, this step will not be required after the release, but it will be needed if you want to modify the code and use the modified version.

The piece of code below, add the location of the new MavenRepository that is located in the path **/maven/local/repository**; this is the same path use in the container stating.

This step imports the required modules to execute the code. All these packages come from the previous Maven Instalation

The imported libraries are:

Module | Java's | Scala's | Description
:----- | -------------: | --------------: | :----------
wayang-core | 8, 11 | 2.11, 2.12 | provides core data structures and the optimizer (required)
wayang-basic | 8, 11 | 2.11, 2.12 | provides common operators and data types for your apps (recommended)
wayang-api-scala-java | 8, 11 | 2.11, 2.12 | provides an easy-to-use Scala and Java API to assemble wayang plans (recommended)
wayang-java | 8, 11 | 2.11, 2.12 | adapters for [Java Stream](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.html) processing platforms
wayang-spark | 8, 11 | 2.11, 2.12 | adapters for [Apache Spark](https://spark.apache.org) processing platforms
wayang-flink | 8, 11 | 2.11, 2.12 | adapters for [Apache Flink](https://flink.apache.org) processing platforms
hadoop-common | 8,11 | - | Hadoop-commons is required because the lack of the Environment Variable **HADOOP_HOME**
log4j-core | 8,11 | - | Logggin library to manipulate the logs

In [1]:
import $ivy.`org.apache.wayang::wayang-api-scala-java:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-core:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-basic:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-java:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang::wayang-spark:0.6.1-SNAPSHOT`
import $ivy.`org.apache.hadoop:hadoop-common:2.8.5`
import $ivy.`org.apache.logging.log4j:log4j-core:2.14.0`

import org.apache.wayang.api._
import org.apache.wayang.core.api.Configuration
import org.apache.wayang.core.api.WayangContext
import org.apache.wayang.core.function.ExecutionContext
import org.apache.wayang.core.function.FunctionDescriptor
import org.apache.wayang.core.plugin.Plugin
import org.apache.wayang.core.util.{Tuple => WayangTuple, WayangCollections}
import org.apache.wayang.java.Java
import org.apache.wayang.spark.Spark
import java.io.File
import java.util.ArrayList
import java.util.Arrays
import java.util.{Collection => JavaCollection}
import java.util.List
import scala.math.{exp, abs, max}
import scala.collection.JavaConversions._

//Logging change the level to INFO
import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

Configurator.setRootLevel(Level.INFO);

import $ivy.$                                                        

import $ivy.$                                             

import $ivy.$                                              

import $ivy.$                                             

import $ivy.$                                               

import $ivy.$                                      

import $ivy.$                                           


import org.apache.wayang.api._

import org.apache.wayang.core.api.Configuration

import org.apache.wayang.core.api.WayangContext

import org.apache.wayang.core.function.ExecutionContext

import org.apache.wayang.core.function.FunctionDescriptor

import org.apache.wayang.core.plugin.Plugin

import org.apache.wayang.core.util.{Tuple => WayangTuple, WayangCollections}

import org.apache.wayang.java.Java

import org.apache.wayang.spark.Spark

import java.io.File

import java.util.ArrayList

import java.util.Arrays

import java.util.{Collection => JavaCollection}

import j

---

Here we include all the classes that has been used in the code

In [2]:
class Transform(var features: Int) 
    extends FunctionDescriptor.SerializableFunction[String, Array[Double]] {

  override def apply(line: String): Array[Double] = {
    val pointStr: Array[String] = line.split(",")
    val point: Array[Double] = Array.ofDim[Double](features + 1)
    for (i <- 0 until pointStr.length) {
      point(i) = pointStr(i).toDouble
    }
    point
  }

}

defined class Transform

In [3]:
class ComputeLogisticGradient
    extends FunctionDescriptor.ExtendedSerializableFunction[Array[Double], Array[Double]] {

  var weights: Array[Double] = _

  override def apply(point: Array[Double]): Array[Double] = {
    val gradient: Array[Double] = Array.ofDim[Double](point.length)
    var dot: Double = 0
    for (j <- 0 until weights.length) dot += weights(j) * point(j + 1)
    for (j <- 0 until weights.length)
      gradient(j + 1) = ((1 / (1 + exp(-1 * dot))) - point(0)) * point(j + 1)
    //counter for the step size required in the update
    gradient(0) = 1
    gradient
  }

  override def open(executionContext: ExecutionContext): Unit = {
    this.weights = executionContext
      .getBroadcast("weights")
      .iterator()
      .next()
      .asInstanceOf[Array[Double]]
  }

}

defined class ComputeLogisticGradient

In [4]:
class Sum
    extends FunctionDescriptor.SerializableBinaryOperator[Array[Double]] {

  override def apply(o: Array[Double], o2: Array[Double]): Array[Double] = {
    val g1: Array[Double] = o
    val g2: Array[Double] = o2
    if (//samples came from one partition only
        g2 == null) g1
    if (//samples came from one partition only
        g1 == null) g2
    val sum: Array[Double] = Array.ofDim[Double](g1.length)
    
    //count
    sum(0) = g1(0) + g2(0)
    for (i <- 1 until g1.length) sum(i) = g1(i) + g2(i)
    sum
  }

}

defined class Sum

In [5]:
class WeightsUpdate
    extends FunctionDescriptor.ExtendedSerializableFunction[Array[Double], Array[Double]] {

  var weights: Array[Double] = _

  var current_iteration: Int = _

  var stepSize: Double = 1

  var regulizer: Double = 0

  def this(stepSize: Double, regulizer: Double) = {
    this()
    this.stepSize = stepSize
    this.regulizer = regulizer
  }

  override def apply(input: Array[Double]): Array[Double] = {
    val count: Double = input(0)
    val alpha: Double = (stepSize / (current_iteration + 1))
    val newWeights: Array[Double] = Array.ofDim[Double](weights.length)
    for (j <- 0 until weights.length) {
      newWeights(j) = (1 - alpha * regulizer) * weights(j) - alpha * (1.0 / count) * input(
          j + 1)
    }
    newWeights
  }

  override def open(executionContext: ExecutionContext): Unit = {
    this.weights = executionContext
      .getBroadcast("weights")
      .iterator()
      .next()
      .asInstanceOf[Array[Double]]
    this.current_iteration = executionContext.getCurrentIteration
  }

}

defined class WeightsUpdate

In [6]:
class ComputeNorm
    extends FunctionDescriptor.ExtendedSerializableFunction[Array[Double], (Double, Double)] {

  var previousWeights: Array[Double] = _

  override def apply(weights: Array[Double]): (Double, Double) = {
    var normDiff: Double = 0.0
    var normWeights: Double = 0.0
    for (j <- 0 until weights.length) {
      normDiff += abs(weights(j) - previousWeights(j))
      normWeights += abs(weights(j))
    }
      
    (normDiff, normWeights)
  }

  override def open(executionContext: ExecutionContext): Unit = {
    this.previousWeights = executionContext
      .getBroadcast("weights")
      .iterator()
      .next()
      .asInstanceOf[Array[Double]]
  }

}

defined class ComputeNorm

In [7]:
class LoopCondition(var accuracy: Double, var max_iterations: Int)
    extends FunctionDescriptor.ExtendedSerializablePredicate[JavaCollection[(Double, Double)]] {

  private var current_iteration: Int = _

  override def test(collection: JavaCollection[(Double, Double)]): Boolean = {
    val input: (Double, Double) = WayangCollections.getSingle(collection)
    println("Running iteration: " + current_iteration)
    (input._1 < accuracy * max(input._2, 1.0) || current_iteration > max_iterations)
  }

  override def open(executionContext: ExecutionContext): Unit = {
    this.current_iteration = executionContext.getCurrentIteration
  }

}

defined class LoopCondition

In [8]:
/**
  * This class executes a stochastic gradient descent optimization on Rheem.
  */
class SGDImpl(plugins: Array[Plugin]) {

  def apply(confFile: Configuration, datasetUrl: String, datasetSize: Int, features: Int, maxIterations: Int, accuracy: Double, sampleSize: Int): Array[Double] = {
    // Initialize the builder.
    val context = new WayangContext(confFile)
    for (plugin <- this.plugins) {
      context.withPlugin(plugin)
    }
    val planBuilder = new PlanBuilder(context)
      
    // Create initial weights.
    val weights: List[Array[Double]] = Arrays.asList(Array.ofDim[Double](features))
      
    val weightsBuilder: DataQuanta[Array[Double]] =
      planBuilder.loadCollection(weights).withName("init weights")
      
    // Load and transform the data.
    val transformBuilder: DataQuanta[Array[Double]] = planBuilder
      .readTextFile(datasetUrl).withName("source")
      .mapJava(new Transform(features)).withName("transform")
      
    // Do the SGD
    val loop: DataQuanta[Array[Double]] = weightsBuilder.doWhileJava(
      new LoopCondition(accuracy, maxIterations),
      (w) => {
        var newWeightsDataset: DataQuanta[Array[Double]] =
          transformBuilder
            .sample(sampleSize, datasetSize).withBroadcast(w, "weights")
            .mapJava(new ComputeLogisticGradient()).withBroadcast(w, "weights").withName("compute")
            .reduceJava(new Sum()).withName("reduce")
            .mapJava(new WeightsUpdate()).withBroadcast(w, "weights").withName("update")
          
        var convergenceDataset: DataQuanta[(Double, Double)] = 
                 newWeightsDataset.mapJava(new ComputeNorm()).withBroadcast(w, "weights")
          
        new WayangTuple(newWeightsDataset, convergenceDataset)
      },
      maxIterations
    )
      
    WayangCollections.getSingleOrNull(loop.collect())
    
  }

}

defined class SGDImpl

In [9]:
val inputFile = new File("files/HIGGS.csv").toURI().toString()
val confFile = new Configuration(new File("files/wayang_sgd.properties").toURI().toString())

new SGDImpl(Array(Spark.basicPlugin, Java.basicPlugin)).apply(confFile, inputFile, 11000000, 28, 1000, 1, 1)

15:19:55.362 [scala-interpreter-1] INFO  org.apache.wayang.core.api.Job - Preparing plan...
15:19:55.491 [scala-interpreter-1] INFO  org.apache.wayang.core.api.Job - Estimating cardinalities and execution load...
15:19:56.370 [scala-interpreter-1] WARN  org.apache.wayang.core.api.configuration.FunctionalKeyValueProvider - Creating fallback load estimator for TransformationDescriptor[ammonite.$sess.cmd1$Helper$Transform@7fe82571].
15:19:56.406 [scala-interpreter-1] WARN  org.apache.wayang.core.api.configuration.FunctionalKeyValueProvider - Creating fallback load estimator for PredicateDescriptor[ammonite.$sess.cmd6$Helper$LoopCondition@3af98741].
15:19:56.418 [scala-interpreter-1] WARN  org.apache.wayang.core.api.configuration.FunctionalKeyValueProvider - Creating fallback load estimator for TransformationDescriptor[ammonite.$sess.cmd2$Helper$ComputeLogisticGradient@72328af6].
15:19:56.428 [scala-interpreter-1] WARN  org.apache.wayang.core.api.configuration.FunctionalKeyValueProvider - 

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.30/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.10/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/03/04 15:20:03 INFO SparkContext: Running Spark version 3.1.2
22/03/04 15:20:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/04 15:20:03 INFO ResourceUtils: ==============================================================
22/03/04 15:20:03 INFO 

15:20:05.040 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Having SparkExecutor[1] execute ExecutionStage[T[SparkCollectionSource[convert out@JavaMap[transform]]]]:
> In  CollectionChannel => SparkCollectionSource[convert out@JavaMap[transform]]
>     SparkCollectionSource[convert out@JavaMap[transform]] => RddChannel => SparkCache[convert out@JavaMap[transform]]
> Out SparkCache[convert out@JavaMap[transform]] => RddChannel
15:20:05.387 [scala-interpreter-1] WARN  org.apache.wayang.spark.execution.SparkExecutor - Execution of T[SparkCollectionSource[convert out@JavaMap[transform]]] took suspiciously long (0:00:00.345).


22/03/04 15:20:05 INFO SparkContext: Starting job: foreachPartition at SparkCacheOperator.java:62
22/03/04 15:20:05 INFO DAGScheduler: Got job 0 (foreachPartition at SparkCacheOperator.java:62) with 4 output partitions
22/03/04 15:20:05 INFO DAGScheduler: Final stage: ResultStage 0 (foreachPartition at SparkCacheOperator.java:62)
22/03/04 15:20:05 INFO DAGScheduler: Parents of final stage: List()
22/03/04 15:20:05 INFO DAGScheduler: Missing parents: List()
22/03/04 15:20:05 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[1] at filter at RddChannel.java:91), which has no missing parents
22/03/04 15:20:05 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 4.6 KiB, free 880.5 MiB)
22/03/04 15:20:05 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 2.4 KiB, free 880.5 MiB)
22/03/04 15:20:05 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 3164d0df1529:46189 (size: 2.4 KiB, free: 880.5 MiB)
22/03/04 15:

15:20:07.034 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Executed 2 items in 0:00:01.645 (estimated (0:00:04.859 .. 0:00:04.898, p=81.23%)).
15:20:07.034 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Executed ExecutionStage[T[SparkCollectionSource[convert out@JavaMap[transform]]]] in 0:00:01.993 (1993 ms).
15:20:07.035 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Having JavaExecutor[0] execute ExecutionStage[T[JavaDoWhile[3->2, id=3e27289b]]]:
> In  CollectionChannel => JavaDoWhile[3->2, id=3e27289b]
> In  CollectionChannel => JavaDoWhile[3->2, id=3e27289b]
> In  CollectionChannel => JavaDoWhile[3->2, id=3e27289b]
> Out JavaDoWhile[3->2, id=3e27289b] => StreamChannel
> Out JavaDoWhile[3->2, id=3e27289b] => StreamChannel
15:20:07.036 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Executed 1 items in 0:00:00.001 (estimated (0:00:00.002 

...

22/03/04 15:20:07 INFO MapPartitionsRDD: Removing RDD 1 from persistence list
22/03/04 15:20:07 INFO BlockManager: Removing RDD 1


15:20:07.427 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Having JavaExecutor[0] execute ExecutionStage[T[JavaLocalCallbackSink[collect()]]]:
> In  StreamChannel => JavaLocalCallbackSink[collect()]
15:20:07.428 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Executed 1 items in 0:00:00.000 (estimated (0:00:00.001 .. 0:00:00.001, p=90.00%)).
15:20:07.428 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Executed ExecutionStage[T[JavaLocalCallbackSink[collect()]]] in 0:00:00.000 (0 ms).
15:20:07.431 [scala-interpreter-1] INFO  org.apache.wayang.core.platform.CrossPlatformExecutor - Executed 11 stages in 0:00:06.968 (6968 ms).
15:20:07.431 [scala-interpreter-1] INFO  org.apache.wayang.core.optimizer.cardinality.CardinalityEstimatorManager - Updating cardinality of JavaMap[transform]'s output 0 from (104,470..115,467, 95.00%) to (110,000..110,000, 100.00%).
15:20:07.432 [scala-inter

22/03/04 15:20:07 INFO SparkUI: Stopped Spark web UI at http://3164d0df1529:4040
22/03/04 15:20:07 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
22/03/04 15:20:07 INFO MemoryStore: MemoryStore cleared
22/03/04 15:20:07 INFO BlockManager: BlockManager stopped
22/03/04 15:20:07 INFO BlockManagerMaster: BlockManagerMaster stopped
22/03/04 15:20:07 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
22/03/04 15:20:07 INFO SparkContext: Successfully stopped SparkContext


15:20:07.723 [scala-interpreter-1] INFO  org.apache.wayang.core.api.Job - StopWatch results:
* Optimization                            - 0:00:05.076
  * Prepare                               - 0:00:00.117
    * Prune&Isolate                       - 0:00:00.029
    * Transformations                     - 0:00:00.088
    * Sanity                              - 0:00:00.000
  * Cardinality&Load Estimation           - 0:00:01.912
    * Create OptimizationContext          - 0:00:00.083
    * Create CardinalityEstimationManager - 0:00:00.001
    * Push Estimation                     - 0:00:01.828
      * Estimate source cardinalities     - 0:00:01.072
  * Create Initial Execution Plan         - 0:00:03.031
    * Enumerate                           - 0:00:02.887
      * Concatenation                     - 0:00:01.583
        * Channel Conversion              - 0:00:01.529
      * Prune                             - 0:00:01.120
    * Pick Best Plan                      - 0:00:00.107
    * Split

inputFile: String = "file:/home/jovyan/work/files/HIGGS.csv"
confFile: Configuration = Configuration[file:/home/jovyan/work/files/wayang_sgd.properties]
res8_2: Array[Double] = Array(
  0.1620724640411348,
  -0.7954137979210483,
  -0.8245163390929532,
  0.7273849291047905,
  0.08607948894141675,
  0.23305201346711177,
  0.2513306789955911,
  -0.033432965293703265,
  0.0,
  0.43554001211901405,
  0.06130203905905599,
  0.6732049581344751,
  1.109276154278117,
  0.6464049914937237,
  0.09814112102492911,
  -0.5814978994038001,
  1.2741122245788574,
  0.2588913381706192,
  0.6657249521807111,
  -0.8592695153674501,
  0.0,
  0.43110926578632486,
  0.4282697499077778,
  0.4949573960856999,
  0.5516440596082104,
  0.4358288941642178,
  0.44888065695173923,
  0.4275738364598869
)